Preparation of the World Ocean Database data for the period and depth of interest.     
The domain is defined in the variable `dom` (tuple).

In [12]:
using NCDatasets
using PhysOcean
using DIVAnd
using PyPlot
using Dates
using Statistics
const plt = PyPlot

using PyCall
colors = PyCall.pyimport("matplotlib.colors")
ccrs = PyCall.pyimport("cartopy.crs")
cfeature = PyCall.pyimport("cartopy.feature")
mticker = PyCall.pyimport("matplotlib.ticker")
coast = cfeature.GSHHSFeature(scale="full");
cartopyticker = PyCall.pyimport("cartopy.mpl.ticker")
lon_formatter = cartopyticker.LongitudeFormatter()
lat_formatter = cartopyticker.LatitudeFormatter()
cmocean = PyCall.pyimport("cmocean")

PyObject <module 'cmocean' from '/home/ctroupin/.julia/conda/3/lib/python3.7/site-packages/cmocean/__init__.py'>

In [13]:
mycolor = "#6667AB"
mycolor2 = "#456A30" # Green treetop
dom = [-20.5, 11.75, 41.25, 67.]
mainproj = ccrs.Mercator(central_longitude=0.5*(dom[1] + dom[2]),
    min_latitude=dom[3], max_latitude=dom[4])
datacrs = ccrs.PlateCarree();

In [14]:
datadir = "/media/ctroupin/My Passport/data/WOD/NorthSea-EMODnetBio"
outputdir = "../data/WOD"
figdir = "../figures/WOD"
varname = "Temperature"
mycolor = "#6667AB"
mycolor2 = "#456A30" # Green treetop
isdir(outputdir) ? @debug("ok") : mkpath(outputdir)
isdir(figdir) ? @debug("ok") : mkpath(figdir);

In [16]:
outputfile = joinpath(outputdir, "temperature_surface_WOD2.nc")

if isfile(outputfile)
    @info("Already read the data from WOD");
    
    @time obsvalue, obslon, obslat, obsdepth, obstime, obsids = loadobs(Float64, outputfile, varname);
    
else
    @info("Reading data from WOD and re-writing in a new netCDF");
    
    #@time obsvalue, obslon, obslat, obsdepth, obstime, obsid = 
    #WorldOceanDatabase.load(Float64, datadir, varname);

    #@time DIVAnd.saveobs(outputfile, 
    #        varname, obsvalue[gooddepth], 
    #        (obslon[gooddepth], obslat[gooddepth], obsdepth[gooddepth], obstime[gooddepth]), 
    #        obsid[gooddepth]);
end

  2.291502 seconds (4.74 M allocations: 313.105 MiB, 8.57% gc time, 89.88% compilation time)


┌ Info: Already read the data from WOD
└ @ Main In[16]:4


([12.770000457763672, 12.65999984741211, 12.380000114440918, 12.550000190734863, 12.100000381469727, 12.640000343322754, 12.6899995803833, 12.739999771118164, 12.170000076293945, 12.5  …  9.729999542236328, 9.779999732971191, 9.779999732971191, 9.720000267028809, 9.680000305175781, 9.680000305175781, 9.640000343322754, 9.369999885559082, 9.369999885559082, 9.300000190734863], [-19.941667556762695, -19.941667556762695, -19.941667556762695, -19.886667251586914, -19.886667251586914, -19.831666946411133, -19.856666564941406, -19.836666107177734, -19.91666603088379, -19.78333282470703  …  -1.9299999475479126, -1.7999999523162842, -1.7999999523162842, -1.5, -1.1699999570846558, -1.1699999570846558, -1.0, -0.6700000166893005, -0.6700000166893005, -0.33000001311302185], [52.59166717529297, 52.59166717529297, 52.59166717529297, 52.599998474121094, 52.599998474121094, 52.606666564941406, 52.60333251953125, 52.61166763305664, 52.58333206176758, 52.61666488647461  …  59.279998779296875, 59.2799987

In [15]:
gooddepth = findall(obsdepth .< 5.);
@info(length(gooddepth));

LoadError: UndefVarError: obsdepth not defined

## Time histogram
Get the years and months for the observations near surface (i.e., shallower than 5 m).

In [17]:
years = Dates.year.(obstime);
months = Dates.month.(obstime);

function count_years_months(year::Array, month::Array)
    
    nyears = maximum(year) - minimum(year) + 1
    @info(nyears)
    yearcount = zeros(nyears)
    monthcount = zeros(12)
    
    ii = 0
    for yyyy in minimum(year):maximum(year)
        ii += 1
        yearcount[ii] = sum(year .== yyyy)
    end
    
    for mm in 1:12
        monthcount[mm] = sum(month .== mm)
    end
       
    return yearcount, monthcount
end

yearcount, monthcount = count_years_months(years, months);

┌ Info: 51
└ @ Main In[17]:7


### Month

In [18]:
fig = plt.figure(figsize=(12, 8))
ax = plt.subplot(111)
plt.bar(1:12, monthcount, color=mycolor)
ax.set_xticks(collect(1:12))
ax.set_xticklabels(Dates.monthname.(1:12))
ax.set_xlim(0.5, 12.5)
ax.spines["right"].set_visible(false)
ax.spines["top"].set_visible(false)
ax.spines["bottom"].set_visible(false)

fig.autofmt_xdate()
ax.set_title("Monthly distribution of observations from WOD")
plt.savefig(joinpath(figdir, "WOD_time_histogram_month"), dpi=300, bbox_inches="tight")
#plt.show()
plt.close()

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /home/ctroupin/.julia/packages/PyCall/3fwVL/src/numpy.jl:67


### Month polar

In [19]:
N = 12
theta = LinRange(0, 2 * π - π/6, N)
width = (1.8 * π) / N
fig = plt.figure(figsize=(10, 10))
ax = plt.subplot(111, polar=true)
ax.set_thetagrids(collect(0:30:330), Dates.monthname.(1:12))
ax.set_theta_zero_location("N")
bars = ax.bar(theta, monthcount, width=width, color=mycolor)
plt.savefig(joinpath(figdir, "WOD_time_histogram_month_polar"), dpi=300, bbox_inches="tight")
plt.close()

### Year

In [20]:
fig = plt.figure(figsize=(12, 8))
ax = plt.subplot(111)
plt.bar(minimum(years):maximum(years), yearcount, color=mycolor)
ax.spines["right"].set_visible(false)
ax.spines["top"].set_visible(false)
ax.spines["bottom"].set_visible(false)
ax.set_title("Yearly distribution of observations from WOD")

plt.savefig(joinpath(figdir, "WOD_time_histogram_year"), dpi=300, bbox_inches="tight")
# plt.show()
plt.close()

## Spatial distribution

In [23]:
fig = plt.figure(figsize=(12, 12))
ax = plt.subplot(111, projection=mainproj)
ax.plot(obslon, obslat, "ko", markersize=1, transform=datacrs)
ax.add_feature(coast, facecolor="#363636", edgecolor="k", zorder=5)

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=true,
                      linewidth=.5, color="gray", alpha=1, linestyle="--")
gl.top_labels = false
gl.right_labels = false
gl.xformatter = lon_formatter
gl.yformatter = lat_formatter
gl.xlabel_style = Dict("size" => 10)
gl.ylabel_style = Dict("size" => 10)
ax.set_title("Locations of the observations")
plt.savefig(joinpath(figdir, "location_obs"), dpi=300, bbox_inches="tight")
plt.show()
#plt.close()

LoadError: PyError ($(Expr(:escape, :(ccall(#= /home/ctroupin/.julia/packages/PyCall/3fwVL/src/pyfncall.jl:43 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))) <class 'AttributeError'>
AttributeError("'NoneType' object has no attribute 'axes'")
  File "/home/ctroupin/.julia/conda/3/lib/python3.7/site-packages/matplotlib/pyplot.py", line 729, in savefig
    res = fig.savefig(*args, **kwargs)
  File "/home/ctroupin/.julia/conda/3/lib/python3.7/site-packages/matplotlib/figure.py", line 2180, in savefig
    self.canvas.print_figure(fname, **kwargs)
  File "/home/ctroupin/.julia/conda/3/lib/python3.7/site-packages/matplotlib/backend_bases.py", line 2065, in print_figure
    **kwargs)
  File "/home/ctroupin/.julia/conda/3/lib/python3.7/site-packages/matplotlib/backends/backend_agg.py", line 527, in print_png
    FigureCanvasAgg.draw(self)
  File "/home/ctroupin/.julia/conda/3/lib/python3.7/site-packages/matplotlib/backends/backend_agg.py", line 388, in draw
    s